# CNBC Crawler

Basierend auf den Inhalten, die beim Aufruf der Seite https://www.cnbc.com/search/?query=boeing&qsearchterm=boeing geladen werden, wird die Schnittstelle direkt mittels Requests aufgerufen um die Daten zu laden. 

Der untenstehende Code ruft direkt api.queryly.com mit den passenden Kennwerten auf. Um das Netzwerk nicht zu belasten und eine Sperre inkaufnehemen zu müssen, wird der Code in Intervallen aufgerufen. Dabei wird nach jedem mal laden von 10 Inhalten, eine Pause zwischen 2-10 Sekunden gehalten.

![alt text](screenshots/1.png "1")

In [24]:
import requests
import json
import time
import random
import math

# Funktion zum Speichern der Daten
def save_data(all_data, filename='lists/CNBC_articles.json'):
    with open(filename, 'w') as file:
        json.dump(all_data, file, indent=4)
    print("Daten wurden erfolgreich zwischengespeichert.")

# URL und Basis-Parameter für die GET-Anfrage
url = "https://api.queryly.com/cnbc/json.aspx"
queryly_key = "31a35d40a9a64ab3"
batch_size = 10  # Stellen Sie sicher, dass dies der Batchsize Ihrer Anfrage entspricht
params = {
    "queryly_key": queryly_key,
    "query": "boeing",
    "batchsize": batch_size,
    "callback": "",
    "showfaceted": "false",
    "timezoneoffset": "-60",
    "facetedfields": "formats",
    "facetedkey": "formats|",
    "facetedvalue": "!Press Release|",
    "sort": "date",
    "additionalindexes": "4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"
}

# Initialanfrage, um die Gesamtzahl der Ergebnisse zu ermitteln
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    total_results = data['metadata']['totalresults']
    total_requests_needed = math.ceil(total_results / batch_size)

    all_data = []  # Sammeln aller Daten hier

    for endindex in range(0, total_requests_needed * batch_size, batch_size):
        # Parameter aktualisieren für jede Anfrage
        params['endindex'] = endindex
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data['results'])
            print(f"{int(endindex/batch_size)+1} von {total_requests_needed} Durchgängen ausgeführt. {len(all_data)} Artikel geladen.")
            
            # Speichern der Daten nach jeweils 100 geladenen Ergebnissen
            if len(all_data) % 100 == 0 or (int(endindex/batch_size)+1) == total_requests_needed:
                save_data(all_data)

            # Warten für eine zufällige Zeit zwischen 2 und 10 Sekunden
            wait_time = random.randint(2, 10)
            print(f"Warte {wait_time} Sekunden...")
            time.sleep(wait_time)
        else:
            print("Fehler bei der Anfrage:", response.status_code)
            break

    # Speichern der gesammelten Daten in einer Datei
    with open('lists/CNBC_articles.json', 'w') as file:
        json.dump(all_data, file, indent=4)

    print("Gesamte Daten wurden erfolgreich heruntergeladen und gespeichert.")
else:
    print("Initialanfrage fehlgeschlagen:", response.status_code)


1 von 1193 Durchgängen ausgeführt. 10 Artikel geladen.
Warte 6 Sekunden...
2 von 1193 Durchgängen ausgeführt. 20 Artikel geladen.
Warte 4 Sekunden...
3 von 1193 Durchgängen ausgeführt. 30 Artikel geladen.
Warte 8 Sekunden...
4 von 1193 Durchgängen ausgeführt. 40 Artikel geladen.
Warte 10 Sekunden...
5 von 1193 Durchgängen ausgeführt. 50 Artikel geladen.
Warte 4 Sekunden...
6 von 1193 Durchgängen ausgeführt. 60 Artikel geladen.
Warte 6 Sekunden...
7 von 1193 Durchgängen ausgeführt. 70 Artikel geladen.
Warte 5 Sekunden...
8 von 1193 Durchgängen ausgeführt. 80 Artikel geladen.
Warte 5 Sekunden...
9 von 1193 Durchgängen ausgeführt. 90 Artikel geladen.
Warte 4 Sekunden...
10 von 1193 Durchgängen ausgeführt. 100 Artikel geladen.
Daten wurden erfolgreich zwischengespeichert.
Warte 4 Sekunden...
11 von 1193 Durchgängen ausgeführt. 110 Artikel geladen.
Warte 3 Sekunden...


KeyboardInterrupt: 

### Laden der einzelnen Artikel

Da das vorherige Skript eine JSON mit allen Links zu den verfügbaren Artikeln erzeugt, gilt es nun die dort enthaltenen Artikel herunterzuladen.

Die bereits erzeugte JSON enthält für jeden Artikel folgenden Eintrag, mit passendem Link zur Html Datei.

```cn:liveURL"https://www.cnbc.com/2024/03/22/faa-to-step-up-scrutiny-of-united-airlines-after-recent-incidents.html"```


In der HTML der Artikel ist eine JSON eingebettet, die Text des Artikels enthält. 

Die JSON ist dabei in der Variable ``` <script charSet="UTF-8"> window.__s_data = {``` enthalten.
    

![alt text](screenshots/2.png "2")

Mittels BeautifulSoup und einer RegEx suche nach der genannten ```window.__s_data``` Variablen, wird die enthaltene JSNON extrahiert.

In [36]:
import requests
from bs4 import BeautifulSoup
import json
import re

# URL des Artikels
url = "https://www.cnbc.com/2024/03/22/airbus-says-its-not-happy-about-issues-at-rival-boeing.html"

# Anfrage an die URL senden und Antwort erhalten
response = requests.get(url)

# Überprüfen, ob die Anfrage erfolgreich war
if response.status_code == 200:
    html_content = response.text  # HTML-Inhalt der Antwort

    # Erstellen einer BeautifulSoup-Instanz und Parsen des HTML-Inhalts
    soup = BeautifulSoup(html_content, 'lxml')

    # Suche nach dem <script> Tag, der die gewünschten JSON-Daten enthält
    script_tag = soup.find('script', string=re.compile('window\.__s_data'))

    if script_tag:
        # Extrahieren des JavaScript-Codes als Text
        script_content = script_tag.string

        # Suchen des JSON-Teils im Script-Content mit einem regulären Ausdruck
        json_data_match = re.search(r'window\.__s_data\s*=\s*(\{.*?\});', script_content, re.DOTALL)

        if json_data_match:
            json_data_str = json_data_match.group(1)  # JSON als String

            # Umwandlung des String in ein Python-Dictionary
            json_data = json.loads(json_data_str)

            # Speichern der JSON-Daten in einer Datei
            with open('lists/extracted_json_data.json', 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, ensure_ascii=False, indent=4)
            
            print("Die JSON-Daten wurden erfolgreich gespeichert.")
        else:
            print("JSON-Daten konnten im Script-Tag nicht gefunden werden.")
    else:
        print("Entsprechender <script> Tag nicht gefunden.")
else:
    print(f"Fehler beim Abrufen der Seite: Statuscode {response.status_code}")


Die JSON-Daten wurden erfolgreich gespeichert.
